In [124]:
import os
import subprocess
import numpy as np
import pandas as pd
from astropy.io import ascii
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import fits
from photutils import CircularAperture
from photutils import aperture_photometry
from astropy import wcs

In [2]:
# diffDir = '/media/mj1e16/PP AV-TV/properDiff/'
# medianDir = '/media/mj1e16/PP AV-TV/medians/'
# imageDir = '/media/mj1e16/PP AV-TV/keplerCal/'

In [6]:
diffDir = '/data/mj1e16/kepler/properDiff/'
medianDir = '/data/mj1e16/kepler/medians/'
imageDir = '/data/mj1e16/kepler/cal/'

In [7]:
diffImages = os.listdir(diffDir)
diffImages = [x for x in diffImages if '.fits' in x]

medianImages = os.listdir(medianDir)
medianImages = [x for x in medianImages if '.fits' in x]

images = os.listdir(imageDir)
images = [x for x in images if '.fits' in x]

In [8]:
unmatchedObjects = ascii.read('/home/mj1e16/outTables15/firstPassTransients15.csv')

In [9]:
xpos = np.array(unmatchedObjects['X_IMAGE'])
ypos = np.array(unmatchedObjects['Y_IMAGE'])
imname = np.array(unmatchedObjects['NAME'])
index = np.array(unmatchedObjects['index'])
allnames = np.unique(imname)

In [131]:
def load_wcs_from_file(filename,pixCoord,ext=0):
    # Load the FITS hdulist using astropy.io.fits
    hdulist = fits.open(filename)
    print(filename)
    # Parse the WCS keywords in the primary HDU
    print(int(ext))
    w = wcs.WCS(hdulist[int(ext)].header)

    # Print out the "name" of the WCS, as defined in the FITS header
    #print(w.wcs.name)

    # Print out all of the settings that were parsed from the header
    w.wcs.print_contents()

    # Three pixel coordinates of interest.
    # Note we've silently assumed a NAXIS=2 image here.
    # Note also that the pixel coordinates are pairs of [X, Y], 
    # and since WCS built from FITS header automatically has
    # origin set to 1, [0, 0] is actually not inside the image.
    pixcrd = np.array(pixCoord, dtype=np.float_)

    # Convert pixel coordinates to world coordinates
    # The second argument is "origin" -- in this case we're declaring we
    # have 1-based (Fortran-like) coordinates.
    world = w.wcs_pix2world(pixcrd, 1)
    #print(world)
    return(world)

In [91]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType,colour='lightgreen',radius='5'):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'global color='+colour+'\nimage\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},'+radius+')\n'.format(xcoords[x],ycoords[x])
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)

In [10]:
def findName(image,diffDir=diffDir,medianDir=medianDir):
    hdu = fits.open(diffDir+image)
    head = hdu[0].header['DIFFCMD']
    locator1 = '/data/mj1e16/kepler/cal/'
    locator2 = ']'
    loc1 = head.find(locator1) + len(locator1)
    loc2 = head.find(locator2) + 1
    imname = head[loc1:loc2]
    
    channellocator1 = 'channel' 
    channellocator2 = '_ccd'
    ccdlocator1 = 'ccd'
    ccdlocator2 = '.fits'
    loc1 = image.find(channellocator1) + len(channellocator1)
    loc2 = image.find(channellocator2)
    channel = image[loc1:loc2]
    loc1 = image.find(ccdlocator1) + len(ccdlocator1)
    loc2 = image.find(ccdlocator2)
    ccd = image[loc1:loc2]
    
    medianName = 'meidian_{}_{}.fits'.format(channel,ccd)
    results = [imname,medianName]
    return results

In [156]:
def findMatchesNoCondition(image,diffname,tab,match,df,bigdf,max_sep = 2.5 * u.arcsec,magsurplus=-3):
    
    
    number1 = match['NUMBER'] # original image
    ra1 = match['RA']
    dec1 = match['DEC']

    number2 = tab['index'] # difference image
    ra2 = tab['RA']
    dec2 = tab['DEC']    
    

    cat1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)
    cat2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)
    
    idx, d2d, d3d = cat2.match_to_catalog_sky(cat1)
    sep_constraint = d2d < max_sep
    objectsFound = number2[sep_constraint]
    matchdf = df[df['ind'].isin(objectsFound)]
    matchdf.sort_values(['Y_IMAGE'])
    ognumber = matchdf['ind'].tolist()
    ogflux = matchdf['FLUX_MAX'].tolist()
    matchTab = Table.from_pandas(matchdf)

    #makeDS9RegFile(matchTab,'foundinBoth','sex',colour='blue')
    
    idx, d2d, d3d = cat1.match_to_catalog_sky(cat2)
    sep_constraint = d2d < max_sep
    #istrue = [x for x in sep_constraint if x == True]
    objectsFound = number1[sep_constraint]
    origdf = bigdf[bigdf['NUMBER'].isin(objectsFound)]
    origdf.sort_values(['Y_IMAGE'])
    origflux = origdf['FLUX_MAX'].tolist()
    
    fluxdiff = [(x-y) for x,y in zip(origflux,ogflux)]
    magdiff = [-2.5*np.log10(x/y) for x,y in zip(origflux,fluxdiff)]
    print(magdiff)
    goodNums = [x for x,y in zip(ognumber,magdiff) if y < magsurplus]
    
    finaldf = matchdf[matchdf['ind'].isin(goodNums)]
    
    return finaldf

In [ ]:
image2diff = {}
image2median = {}
for x in diffImages:
    results = findName(x)
    image2diff[results[0]] = x
    image2median[results[0]] = results[1]
    
diff = [image2diff[x] for x in imname]
median = [image2median[x] for x in imname]

unmatchedObjects['diffImage'] = diff
unmatchedObjects['medianImage'] = median

In [136]:
name = imname[0]
diffname = image2diff[name]
df = unmatchedObjects.to_pandas()
mindf = df[df['NAME'] == name]
minitab = Table.from_pandas(mindf)
matchtab = ascii.read('/data/mj1e16/kepler/sexKeplerCal/'+name+'.csv')
#mediantab = ascii.read('/data/mj1e16/kepler/sexKeplerMedian/'+image2median[name]+'.csv')
bigdf = matchtab.to_pandas()
#meddf = mediantab.to_pandas()

In [146]:
df

,ind,NUMBER,MAG_BEST,FLUX_MAX,X_IMAGE,Y_IMAGE,ELONGATION,ELLIPTICITY,RA,DEC,NAME,index,diffImage,medianImage
0,0,4,17.5644,511.4224,1014.9202,20.2358,1.084,0.077,288.808321,41.572243,kplr2012310200152_ffi-cal.fits[58],0,diff_imNo23_channel66_ccd58.fits,meidian_66_58.fits
1,6,26,16.7801,662.9155,302.7606,994.0505,1.121,0.108,290.475188,41.113748,kplr2012310200152_ffi-cal.fits[58],6,diff_imNo23_channel66_ccd58.fits,meidian_66_58.fits
2,8,29,15.0461,1526.7550,418.6525,979.0203,1.031,0.030,290.320998,41.057570,kplr2012310200152_ffi-cal.fits[58],8,diff_imNo23_channel66_ccd58.fits,meidian_66_58.fits
3,9,32,17.2988,1266.7620,419.8381,987.3757,1.269,0.212,290.326257,41.049136,kplr2012310200152_ffi-cal.fits[58],9,diff_imNo23_channel66_ccd58.fits,meidian_66_58.fits
4,14,53,17.6437,1172.9480,823.7241,908.1454,1.160,0.138,289.769297,40.876473,kplr2012310200152_ffi-cal.fits[58],14,diff_imNo23_channel66_ccd58.fits,meidian_66_58.fits
5,15,59,17.6208,402.1461,771.9370,900.6266,1.294,0.227,289.826259,40.915028,kplr2012310200152_ffi-cal.fits[58],15,diff_imNo23_channel66_ccd58.fits,meidian_66_58.fits
6,16,64,17.9043,1005.5340,37.0523,894.7243,1.104,0.095,290.722688,41.365745,kplr2012310200152_ffi-cal.fits[58],16,diff_imNo23_channel66_ccd58.fits,meidian_66_58.fits
7,19,72,18.0369,282.3715,760.0580,857.7233,1.081,0.075,289.806025,40.961755,kplr2012310200152_ffi-cal.fits[58],19,diff_imNo23_channel66_ccd58.fits,meidian_66_58.fits
8,24,89,18.1406,494.5627,567.5733,779.4306,1.315,0.240,289.977722,41.151238,kplr2012310200152_ffi-cal.fits[58],24,diff_imNo23_channel66_ccd58.fits,meidian_66_58.fits
9,25,92,17.7215,565.4743,585.9466,777.1949,1.152,0.132,289.953432,41.142105,kplr2012310200152_ffi-cal.fits[58],25,diff_imNo23_channel66_ccd58.fits,meidian_66_58.fits


In [159]:
matches = findMatchesNoCondition(imageDir+name,diffDir+diffname,minitab,matchtab,mindf,bigdf,max_sep=5*u.arcsec)

[-0.05620325995845364, nan, nan, -0.7271288192655224, -0.014472199334716142, -0.05300994831146413, -1.4571815313378544, nan, -0.02180722264547895, -0.011935830200145239, -0.017960166210759618, -0.14517939577093023, -0.022554623422719782, -0.009808813321782576, -1.6606910087271756]


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in log10


In [16]:
ascii.write(unmatchedObjects,'/home/mj1e16/outTables15/transientsWnames.csv',format='csv',fast_writer=False)